In [1]:
import yaml
import os
import glob
import json
from datetime import datetime
from bs4 import BeautifulSoup
import copy

prj = r"C:\Users\mmcmanus\Documents\Working\models\FIA Darlington\AmiteWatershed_2016Event_WithDarlingtonReservoir\AmiteWatershed_2016Event.prj"
prj_dir, prj_file_tail = os.path.split(prj)
prj_name = prj_file_tail.split(".")[0]

# coordinate_system
with open(f'{prj_dir}/{prj_name}.projection', "r") as f:
     coordinate_system = f.read()

with open(prj, "r") as f:
     lines = f.readlines()
lines = [s.strip('\n') for s in lines]
# lines

In [32]:

keyValueList = copy.deepcopy(lines)  

# get fia version, author, and date
proj_fia_version        = None
proj_created_date_unix  = None
proj_created_date       = None
proj_description        = None
proj_author             = None

for i,v in enumerate(lines):
        # print (v)
        if 'Version=' in v:
                proj_fia_version = v.split('=')[-1]
        
        if 'Created=' in v:
                # Timestamp is in milliseconds. Divided by 1000 to get timestamp in seconds
                proj_created_date_unix = int(v.split('=')[-1])/1000
                # Convert to datetime format 
                proj_created_date = datetime.utcfromtimestamp(proj_created_date_unix).strftime("%Y-%m-%d")

        if 'ProjectDescription=' in v:
                proj_description = v.split('=')[-1]

        if 'Created By=' in v:
                proj_author = v.split('=')[-1]
        
# Find the first wanted header to start parsing Map and Manager blocks, get the line number, break the loop.
for i,v in enumerate(lines):
        if (v == 'MapBegin') or (v == 'ManagerBegin'):
                startLine_after_proj_info = i
                break

# Parse the wanted blocks of Manager and Map.
nestList = []
for i,v in enumerate(lines):
        
        if (v == 'ManagerEnd') or (v == 'MapEnd'):
                # If not the beginning of the file, skip a blank line (+1) for the start of the subList.
                if len(nestList) > 0:
                        nestList.append(lines[startLine_after_proj_info+1:i])
                else:
                        nestList.append(lines[startLine_after_proj_info:i])
                startLine_after_proj_info = i+1



AsciiSerializer=4
{=hec2.fia.model.TimeWindow
ID=0
FLD=_index
I=-2147483648
FLD=_name
STR=August2016_Event_TimeWindow
FLD=_description
STR=
FLD=_lastModifiedTime
J=1553545442379
FLD=_createdBy
STR=kslover
FLD=_modifiedBy
STR=kslover
FLD=_createdDate
J=1549043901034
FLD=m_startTime
hec.heclib.util.HecTime=61331760,1
FLD=m_endTime
hec.heclib.util.HecTime=61341840,1
FLD=m_lookbackTime
hec.heclib.util.HecTime=61331759,1
}


In [3]:
# Break the nestList into 2 lists each for Maps and Managers by chcking the first value in each subList.
# Only appending the values in each subList from columns [1:4] for mapList and [1:5] for managerList. The others deemed unnecessary.
mapList = []
managerList = []
for subList in nestList:
    if subList[0] == 'MapBegin':
        mapList.append(subList[1:4])
    if subList[0] == 'ManagerBegin':
        managerList.append(subList[1:5])

In [4]:
# mapList

In [5]:
map_kv_list = []
for subList in mapList:
    kv = {
        # Name
        subList[0].split('=')[0] : subList[0].split('=')[-1],
        # Description
        subList[1].split('=')[0] : subList[1].split('=')[-1],
        # Path
        subList[2].split('=')[0] : subList[2].split('=')[-1],  
    }
    map_kv_list.append(kv)
# map_kv_list

In [6]:
# spatial extent - ManagerList - Class=hec2.fia.model.TerrainModelManager - File=watershed/terrain grids/amite_ws_10ft_dem.tgrid
# Find subList in ManagerList containing Class=hec2.fia.model.TerrainModelManager
# managerList
# get all class types


In [7]:
print (managerList[0])

['Name=Program Order Manager', 'Description=', 'File=shared/programOrder.xml', 'Class=hec2.model.ProgramOrderManager']


In [8]:
manager_kv_list = []
for subList in managerList:
    kv = {
        # Name
        subList[0].split('=')[0] : subList[0].split('=')[-1],
        # Description
        subList[1].split('=')[0] : subList[1].split('=')[-1],
        # File
        subList[2].split('=')[0] : subList[2].split('=')[-1],
        # Class
        subList[3].split('=')[0] : subList[3].split('=')[-1].split('.')[-1],   
    }
    manager_kv_list.append(kv)
# manager_kv_list

In [9]:
import pandas as pd
manager_df = pd.DataFrame(manager_kv_list)
# manager_df
map_df = pd.DataFrame(map_kv_list)
# map_df.loc[manager_df.Description == '', 'Description'] = ' '
map_df

,Name,Description,Path
0,Amite_StructureInventory_NAD83_Albers.shp,,maps/Amite_StructureInventory_NAD83_Albers.shp
1,AmiteParishes_NAD83_Albers.shp,,maps/AmiteParishes_NAD83_Albers.shp
2,AmiteHUC8_NAD83_Albers.shp,,maps/AmiteHUC8_NAD83_Albers.shp


In [10]:

#  Update Names to be more descriptive based on Class &/or Description Columns.
manager_df.loc[manager_df.Class == 'AgricultureManager', 'Name'] =  'Agriculture Data: ' + manager_df.loc[manager_df.Class == "AgricultureManager"].Name
manager_df.loc[manager_df.Class == 'Alternative', 'Name'] =  'Alternative: ' + manager_df.loc[manager_df.Class == "Alternative"].Name
manager_df.loc[manager_df.Class == 'BoundaryManager', 'Name'] =  manager_df.loc[manager_df.Class == "BoundaryManager"].Class.str.replace('Manager', ': ') + manager_df.loc[manager_df.Class == "BoundaryManager"].Name
manager_df.loc[manager_df.Class == 'AnalysisGroup', 'Name'] = manager_df.loc[manager_df.Class == "AnalysisGroup"].Description
manager_df.loc[manager_df.Class == 'GridsInundationConfiguration', 'Name'] =  'Grid Configuration: ' + manager_df.loc[manager_df.Class == "GridsInundationConfiguration"].Name
manager_df.loc[manager_df.Class == 'ImpactAreaSetManager', 'Name'] =  "Impact Area Set"
manager_df.loc[manager_df.Class == 'LifeSimModel', 'Name'] =  'LifeSim Model: ' + manager_df.loc[manager_df.Class == "LifeSimModel"].Name
manager_df.loc[manager_df.Class == 'Simulation', 'Name'] =  'Simulation: ' + manager_df.loc[manager_df.Class == "Simulation"].Name
manager_df.loc[manager_df.Class == 'StructureManager', 'Name'] =  'Structures: ' + manager_df.loc[manager_df.Class == "StructureManager"].Name
manager_df.loc[manager_df.Class == 'TerrainModelManager', 'Name'] =  'Terrain: ' + manager_df.loc[manager_df.Class == "TerrainModelManager"].Name
manager_df.loc[manager_df.Class == 'TimeWindow', 'Name'] =  'Time Window: ' + manager_df.loc[manager_df.Class == "TimeWindow"].Name
manager_df.loc[manager_df.Class == 'WarningIssuanceManager', 'Name'] =  'Warning Issuance: ' + manager_df.loc[manager_df.Class == "WarningIssuanceManager"].Name
manager_df.loc[manager_df.Class == 'WatershedConfiguration', 'Name'] =  'Watershed Configuration: ' + manager_df.loc[manager_df.Class == "WatershedConfiguration"].Name

# Include proj_description in study description
manager_df.loc[manager_df.Name == 'Study', 'Description'] =  proj_description

manager_df.loc[manager_df.Description == '', 'Description'] = ' '

In [11]:
# manager_df

In [12]:
# open the model application Json template, del unnecessary keys, update, add, export 
with open(r"model_application_fia_template.json", 'r') as f:
            model_template_json = json.load(f)

# keys to drop from json template
drop_keys = ['_id', 'linked_resources', 'common_parameters', 'common_software_version', 'authors', 
'spatial_extent_resolved', 'spatial_valid_extent_resolved', 'temporal_extent', 'temporal_resolution', 
'spatial_valid_extent', 'common_files_details', 'grid', '__created_at', '__created_by']
for key in drop_keys:
    del model_template_json[key]

In [13]:
model_template_json['description'] = proj_description
model_template_json['purpose'] = proj_description
model_template_json['title'] = f'HEC-FIA Model: {prj_name}'
model_template_json['application_date'] = proj_created_date


model_template_json.keys()

dict_keys(['description', 'keywords', 'title', 'purpose', 'application_date', 'spatial_extent'])

In [14]:
import geopandas as gpd
import yaml
import os

def parse_shp(shp, prj_name, output_dir):
    gmanager_df = gpd.read_file(shp)
    crs = str(gmanager_df.crs)
    gmanager_df = gmanager_df.to_crs(4326)
    wkt = gmanager_df.to_wkt().geometry[0]
    wkt_dict = {}
    wkt_dict["spatial_extent"] = wkt
    wkt_dict["coordinate_system"] = crs

    with open(os.path.join(output_dir, f'{prj_name}_wkt.yml'), 'w+') as f:
        yaml.dump(wkt_dict, f)
    
    return wkt, crs


In [15]:
wkt, crs = parse_shp(r"C:\Users\mmcmanus\Documents\Working\models\FIA Darlington\AmiteWatershed_2016Event_WithDarlingtonReservoir\maps\AmiteHUC8_NAD83_Albers.shp", prj_name, './output')

In [16]:
model_template_json['spatial_extent'] = wkt
model_template_json['grid'] = {
    "coordinate_system": crs,
    "description": "terrain based",
    "shape": "rectangular",
    "dimension": "2D"
}

In [17]:
model_template_json["common_files_details"] = [
        {
            "description": "AgricultureManager",
            "location": "Inventory/Agriculture Data/AgriculturalGrid.tif",
            "title": "Agricultural Manager: AgriculturalGrid"
        }
    ],

In [18]:
map_df

,Name,Description,Path
0,Amite_StructureInventory_NAD83_Albers.shp,,maps/Amite_StructureInventory_NAD83_Albers.shp
1,AmiteParishes_NAD83_Albers.shp,,maps/AmiteParishes_NAD83_Albers.shp
2,AmiteHUC8_NAD83_Albers.shp,,maps/AmiteHUC8_NAD83_Albers.shp


In [19]:
# Match common_files dataframe to template
manager_df = manager_df.rename(columns={"Name": "title", "Description": "description", "File": "location"})
manager_df_clean = manager_df.drop(['Class'], axis=1)
map_df = map_df.rename(columns={"Name": "title", "Description": "description", "Path": "location"})




In [20]:
#Concatenate manager and map dataframes
df = pd.concat([manager_df_clean, map_df], axis=0)

In [21]:
df

,title,description,location
0,Program Order Manager,,shared/programOrder.xml
1,Study,Amite Watershed FIA Model using gridded data f...,study/Study.dbf
2,Computation Point,,study/ComputationPoint.dbf
3,Channel Mod,,study/ChannelMod.dbf
4,Configuration,,study/Condition.dbf
5,Diversion,,study/Diversion.dbf
6,ImpactArea,,study/ImpactArea.dbf
7,Levee,,study/Levee.dbf
8,OtherProject,,study/OtherProject.dbf
9,Reservoir,,study/Reservoir.dbf


In [22]:
# Use Pandas to_dict("records") method, it outputs to the required list of objects format: [{}, {}].
model_template_json["common_files_details"] = df.to_dict('records')
model_template_json["common_files_details"]

[{'title': 'Program Order Manager',
  'description': ' ',
  'location': 'shared/programOrder.xml'},
 {'title': 'Study',
  'description': 'Amite Watershed FIA Model using gridded data from HEC-RAS',
  'location': 'study/Study.dbf'},
 {'title': 'Computation Point',
  'description': ' ',
  'location': 'study/ComputationPoint.dbf'},
 {'title': 'Channel Mod',
  'description': ' ',
  'location': 'study/ChannelMod.dbf'},
 {'title': 'Configuration',
  'description': ' ',
  'location': 'study/Condition.dbf'},
 {'title': 'Diversion', 'description': ' ', 'location': 'study/Diversion.dbf'},
 {'title': 'ImpactArea',
  'description': ' ',
  'location': 'study/ImpactArea.dbf'},
 {'title': 'Levee', 'description': ' ', 'location': 'study/Levee.dbf'},
 {'title': 'OtherProject',
  'description': ' ',
  'location': 'study/OtherProject.dbf'},
 {'title': 'Reservoir', 'description': ' ', 'location': 'study/Reservoir.dbf'},
 {'title': 'StoragePool',
  'description': ' ',
  'location': 'study/StoragePool.dbf'}

In [23]:
# output model application json
output_model_json = os.path.join('./output',f'{prj_name}_model_application.json')
with open(output_model_json, "w") as outfile:
    json.dump(model_template_json, outfile)

In [24]:
# Get Sim Files.
sim_paths = manager_df.loc[manager_df['Class'] =='Simulation', 'location'].to_list()
sim_pathnames = []
for path in sim_paths:
     sim_pathnames.append(os.path.join(prj_dir, path))
# sim_pathnames

# Parse Sims from each sim file get basics and alt file. from alt file get inputs
path = sim_pathnames[0]
# for path in sim_pathnames:
with open(path, 'r') as f:
     sim_file = f.read()

soupData_sim = BeautifulSoup(sim_file, 'xml')
sim_name = soupData_sim.find_all('Name')[0].text
sim_description = soupData_sim.find_all('Description')[0].text
alt_name = soupData_sim.find_all('AlternativeName')[0].text
alt_path = soupData_sim.find_all('AlternativePath')[0].text.split("../")[-1]
event_name = soupData_sim.find_all('EventName')[0].text
timewindow_name = soupData_sim.find_all('TimeWindowName')[0].text
timewindow_path = soupData_sim.find_all('TimeWindowPath')[0].text.split("../")[-1]



In [25]:

with open(os.path.join(prj_dir, alt_path), 'r') as f:
     alt_file = f.read()

soupData_alt = BeautifulSoup(alt_file, 'xml')

alt_description = soupData_alt.find_all('Description')[0].text

wanted_tags = ['Impact Area', 'Inundation Configuration', 'Structrue Inventory', 'Agriculture Inventory', 'Warning Issuance']
# The grid_wanted_tags list comes from the gridlc xml file which is linked in the Alternative XML file.
grid_wanted_tags = ['Inundation Grid', 'Depth Velocity Grid', 'Life Lose Arrival Grid', 'Agriculture Arrival Grid', 'Agriculgure Duration Grid']
alt_kv_list = []

for tag in wanted_tags:
     
     try:
          # Removing spaces from the tag and adding either 'Name' or 'Path', this matches the tag format in the xml file
          # doing this allows the formatting to be controlled by the wanted_tags list. 
          # If more than spaces are needed, should update to use a dictionary of wantedFormattedName:XMLtagName pairs instead of the wanted_tag list.
          tag_formatted = tag.replace(" ", "")
          text_name = soupData_alt.find_all(tag_formatted+'Name')[0].text
          text_path = soupData_alt.find_all(tag_formatted+'Path')[0].text
          
          if '../' in text_path:
               text_path = text_path.split('../')[-1]
          
          alt_kv_list.append({
               'title':tag,
               'description': ' ',
               'location': text_path
          })

          # Innundation Config input files
          if tag == 'Inundation Configuration':

               # open gridslc file
               with open(os.path.join(prj_dir, text_path), 'r') as f:
                    grids_file = f.read()
               
               soupData_grids = BeautifulSoup(grids_file, 'xml')
               
               for tag in grid_wanted_tags:
                    tag_formatted = tag.replace(" ", "")
                    text_path = soupData_grids.find_all(tag_formatted+'Path')[0].text
                    
                    if '../' in text_path:
                         text_path = text_path.split('../')[-1]

                    alt_kv_list.append({
                         'title':tag,
                         'description': ' ',
                         'location': text_path
                    })
     
     except:
          print(f'Missing XML Tag. "<{tag_formatted}Path>" not found in Alternative File: "{os.path.join(prj_dir, alt_path)}"')
          continue
# alt_kv_list




In [26]:
# outputs in dir: runs/{Alt}/{Event}/{TW}. Get *.shp's.
sim_results_dir = os.path.join('runs', alt_name, event_name, timewindow_name)
sim_results_abs_dir = os.path.join(prj_dir, 'runs', alt_name, event_name, timewindow_name)
output_kv_list = []
for result_shp in glob.glob(rf'{sim_results_abs_dir}/*.shp'):
     head, tail = os.path.split(result_shp)
     sim_results_dir_forwardslash = sim_results_dir.replace("\\", "/")
     output_kv_list.append({
          'title': tail.split('.shp')[0],
          'description': ' ',
          'location': f'{sim_results_dir_forwardslash}/{tail}'
     })
output_kv_list

[{'title': 'EconResults',
  'description': ' ',
  'location': 'runs/Alternative_August2016_Event_WithReservoir/August2016_Event_WithReservoir/August2016_Event_TimeWindow/EconResults.shp'},
 {'title': 'LifeLossResults',
  'description': ' ',
  'location': 'runs/Alternative_August2016_Event_WithReservoir/August2016_Event_WithReservoir/August2016_Event_TimeWindow/LifeLossResults.shp'},
 {'title': 'LifeLossResults10AM',
  'description': ' ',
  'location': 'runs/Alternative_August2016_Event_WithReservoir/August2016_Event_WithReservoir/August2016_Event_TimeWindow/LifeLossResults10AM.shp'},
 {'title': 'LifeLossResults10PM',
  'description': ' ',
  'location': 'runs/Alternative_August2016_Event_WithReservoir/August2016_Event_WithReservoir/August2016_Event_TimeWindow/LifeLossResults10PM.shp'},
 {'title': 'LifeLossResults12AM',
  'description': ' ',
  'location': 'runs/Alternative_August2016_Event_WithReservoir/August2016_Event_WithReservoir/August2016_Event_TimeWindow/LifeLossResults12AM.shp'},

In [27]:
# Parameters from Alternative file
parameter_tags = ['Random Seed', 'Confidence', 'Convergence Tolerance', 'Convergence Variables', 'Evacuation Velocity']
parameter_kv_list = []
for tag in parameter_tags:
    try:
        # Removing spaces from the tag and adding either 'Name' or 'Path', this matches the tag format in the xml file
        # doing this allows the formatting to be controlled by the wanted_tags list. 
        # If more than spaces are needed, should update to use a dictionary of wantedFormattedName:XMLtagName pairs instead of the wanted_tag list.
        tag_formatted = tag.replace(" ", "")
        text = soupData_alt.find_all(tag_formatted)[0].text
        parameter_kv_list.append({
            'value': text,
            'parameter': tag
        })        
    except:
        print(f'Missing XML Tag. "<{tag_formatted}>" not found in Alternative File: "{os.path.join(prj_dir, alt_path)}"')
        continue
parameter_kv_list


[{'value': '0.0', 'parameter': 'Random Seed'},
 {'value': '0.9', 'parameter': 'Confidence'},
 {'value': '0.05', 'parameter': 'Convergence Tolerance'},
 {'value': 'Total Damage', 'parameter': 'Convergence Variables'},
 {'value': '25.0', 'parameter': 'Evacuation Velocity'}]

In [28]:
# Open Time Window File and Parse Temporal Extent
with open(os.path.join(prj_dir,timewindow_path), 'r') as f:
    lines = f.readlines()

lines = [s.strip('\n') for s in lines]
# get StartTime & EndTime line index
for i, v in enumerate(lines):
    if 'FLD=m_startTime' in v:
        startTime_idx = i + 1
    if 'FLD=m_endTime' in v:
        endTime_idx = i + 1
startTime_hecTime = lines[startTime_idx].split('=')[-1].split(',')[0]
startTime_utc= int(startTime_hecTime) * 24
startTime_dt = datetime.utcfromtimestamp(startTime_utc)
endTime_hecTime = lines[endTime_idx].split('=')[-1].split(',')[0]
endTime_utc= int(endTime_hecTime) * 24
endTime_dt = datetime.utcfromtimestamp(endTime_utc)

In [29]:
# Open simulation template, drop unwanted keys, update values, output json.
with open(r"simulation_fia_template.json", 'r') as f:
            sim_template_json = json.load(f)

# keys to drop from json template
drop_keys = ['_id', 'linked_resources', 'model_application', 'model_software',
              '__created_at', '__created_by', 'temporal_resolution']
for key in drop_keys:
    del sim_template_json[key]

sim_template_json.keys()

dict_keys(['description', 'type', 'input_files', 'title', 'temporal_extent', 'output_files', 'parameters'])

In [30]:
sim_template_json['title'] = f'HEC-FIA {prj_name} Simulation: {sim_name}'
sim_template_json['description'] = f'HEC-FIA Simulation: {sim_name}, {sim_description}, for project: {prj_name}, using Alternative: {alt_name}.'
sim_template_json['input_files'] = alt_kv_list
sim_template_json['output_files'] = output_kv_list
sim_template_json['parameters'] = parameter_kv_list

In [31]:
# output sim json
output_sim_json = os.path.join('./output',f'{prj_name}_{sim_name}_simulation.json')
with open(output_sim_json, "w") as outfile:
    json.dump(sim_template_json, outfile)